In [1]:
#!pip install tpot autogluon flaml

Random State for reproduceability

In [1]:
dagstuhl_seed=23372

# Data

In [2]:
import numpy as np
import pandas as pd

In [3]:
np.random.seed(dagstuhl_seed) 

In [4]:
df = pd.read_csv('../data/preprocessed.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df_train = pd.read_csv('../data/preprocessed-train.csv')
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]
df_train = df_train.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df_test = pd.read_csv('../data/preprocessed-test.csv')
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]
df_test = df_test.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df.tail()

,label,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
4306,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4307,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4308,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4309,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4310,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
y_train = df_train['label']
X_train = df_train.loc[:, df_train.columns != "label"]

y_test = df_train['label']
X_test = df_train.loc[:, df_train.columns != "label"]

X_train.to_numpy().shape

(3592, 1024)

# AutoML Libraries

### Versions

In [6]:
import sys
sys.version

'3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) \n[GCC 10.3.0]'

In [7]:
import tpot
tpot.__version__

'0.12.1'

In [8]:
from autogluon import tabular
tabular.__version__

'0.8.2'

In [9]:
import flaml
flaml.__version__

'2.1.0'

### Time Limit

In [10]:
time_min = 30
time_sec = time_min * 60

## AutoGluon

In [12]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [13]:
train_data = TabularDataset(df_train)
test_data = TabularDataset(df_test)

In [14]:
%%time
predictor = TabularPredictor(label='label').fit(train_data=train_data, verbosity=4, time_limit=time_sec, presets='best_quality')
# model will decide how many cores to use

No path specified. Models will be saved in: "AutogluonModels/ag-20230925_120315/"
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'included_model_types': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Saving AutogluonModels/ag-2023

CPU times: user 2min 51s, sys: 1min 31s, total: 4min 23s
Wall time: 28min 37s


In [15]:
predictor.evaluate(test_data, silent=False)

Loading: AutogluonModels/ag-20230925_120315/models/CatBoost_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/LightGBMLarge_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/RandomForestEntr_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/WeightedEnsemble_L2/model.pkl
Evaluation: accuracy on test data: 0.9429763560500696
Evaluations on test data:
{
    "accuracy": 0.9429763560500696,
    "balanced_accuracy": 0.5948543187544415,
    "mcc": 0.34841715179951094,
    "roc_auc": 0.8373602945926739,
    "f1": 0.3050847457627119,
    "precision": 0.6923076923076923,
    "recall": 0.1956521739130435
}


{'accuracy': 0.9429763560500696,
 'balanced_accuracy': 0.5948543187544415,
 'mcc': 0.34841715179951094,
 'roc_auc': 0.8373602945926739,
 'f1': 0.3050847457627119,
 'precision': 0.6923076923076923,
 'recall': 0.1956521739130435}

In [16]:
predictor.leaderboard(test_data, silent=True)

Loading: AutogluonModels/ag-20230925_120315/models/LightGBMXT_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/LightGBM_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/RandomForestGini_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/RandomForestEntr_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/CatBoost_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/XGBoost_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/LightGBMLarge_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230925_120315/models/WeightedEnsemble_L2/model.pkl
Model scores:
{'LightGBMX

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L1,0.944367,0.933742,1.760444,1.530567,186.892907,1.760444,1.530567,186.892907,1,True,9
1,ExtraTreesGini_BAG_L1,0.942976,0.932350,0.253660,1.114834,2.713726,0.253660,1.114834,2.713726,1,True,6
2,ExtraTreesEntr_BAG_L1,0.942976,0.932350,0.296739,0.886805,1.812422,0.296739,0.886805,1.812422,1,True,7
3,WeightedEnsemble_L2,0.942976,0.938753,11.978904,19.820915,953.997609,0.008185,0.009452,2.077370,2,True,12
4,RandomForestGini_BAG_L1,0.941586,0.932906,0.282792,0.914254,2.071538,0.282792,0.914254,2.071538,1,True,3
5,RandomForestEntr_BAG_L1,0.941586,0.933463,0.289093,0.856345,1.881582,0.289093,0.856345,1.881582,1,True,4
6,LightGBMXT_BAG_L1,0.941586,0.934855,1.297155,2.449745,63.222678,1.297155,2.449745,63.222678,1,True,1
7,LightGBM_BAG_L1,0.941586,0.934855,1.317754,2.708474,65.453110,1.317754,2.708474,65.453110,1,True,2
8,CatBoost_BAG_L1,0.941586,0.932628,5.173592,9.818150,192.268285,5.173592,9.818150,192.268285,1,True,5
9,LightGBMLarge_BAG_L1,0.938804,0.935134,1.635441,2.300599,117.953072,1.635441,2.300599,117.953072,1,True,11


# FLAML

In [17]:
from flaml import AutoML
from sklearn.metrics import accuracy_score

In [18]:
%%time
automl = AutoML()
automl.fit(X_train, y_train, task="classification", time_budget=time_sec, verbose=9999, ensemble=True)
# uses all cores by default

[flaml.automl.logger: 09-25 12:32:28] {1679} INFO - task = classification
[flaml.automl.logger: 09-25 12:32:28] {1690} INFO - Evaluation method: cv
[flaml.automl.logger: 09-25 12:32:28] {1788} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 09-25 12:32:28] {1900} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 09-25 12:32:28] {2218} INFO - iteration 0, current learner lgbm
[flaml.tune.tune: 09-25 12:32:28] {805} INFO - trial 1 config: {'n_estimators': 4, 'num_leaves': 4, 'min_child_samples': 20, 'learning_rate': 0.09999999999999995, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0}
[flaml.tune.tune: 09-25 12:32:29] {197} INFO - result: {'pred_time': 1.0511825560754945e-05, 'wall_clock_time': 5.228039026260376, 'metric_for_logging': {'pred_time': 1.0511825560754945e-05}, 'val_loss': 0.3371834396384621, 'trained_estimator': <flaml.automl.mod

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.tune.tune: 09-25 12:32:31] {197} INFO - result: {'pred_time': 4.1325934951945794e-05, 'wall_clock_time': 7.452991962432861, 'metric_for_logging': {'pred_time': 4.1325934951945794e-05}, 'val_loss': 0.41041591556958873, 'trained_estimator': <flaml.automl.model.XGBoostSklearnEstimator object at 0x7fe5243b2d00>, 'training_iteration': 0, 'config': {'n_estimators': 4, 'max_leaves': 4, 'min_child_weight': 0.9999999999999993, 'learning_rate': 0.09999999999999995, 'subsample': 1.0, 'colsample_bylevel': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0}, 'config/n_estimators': 4, 'config/max_leaves': 4, 'config/min_child_weight': 0.9999999999999993, 'config/learning_rate': 0.09999999999999995, 'config/subsample': 1.0, 'config/colsample_bylevel': 1.0, 'config/colsample_bytree': 1.0, 'config/reg_alpha': 0.0009765625, 'config/reg_lambda': 1.0, 'experiment_tag': 'exp', 'time_total_s': 1.417604923248291}
[flaml.tune.tune: 09-25 12:32:31] {197} INFO - result: {'pred_tim

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.tune.tune: 09-25 12:32:35] {197} INFO - result: {'pred_time': 4.12732509495236e-05, 'wall_clock_time': 11.125219106674194, 'metric_for_logging': {'pred_time': 4.12732509495236e-05}, 'val_loss': 0.4495247741687022, 'trained_estimator': <flaml.automl.model.XGBoostSklearnEstimator object at 0x7fe6d5e1b3a0>, 'training_iteration': 0, 'config': {'n_estimators': 4, 'max_leaves': 4, 'min_child_weight': 3.815612027960909, 'learning_rate': 0.03859136192132085, 'subsample': 1.0, 'colsample_bylevel': 0.8148474110627004, 'colsample_bytree': 0.9777234800442423, 'reg_alpha': 0.0009765625, 'reg_lambda': 5.525802807180917}, 'config/n_estimators': 4, 'config/max_leaves': 4, 'config/min_child_weight': 3.815612027960909, 'config/learning_rate': 0.03859136192132085, 'config/subsample': 1.0, 'config/colsample_bylevel': 0.8148474110627004, 'config/colsample_bytree': 0.9777234800442423, 'config/reg_alpha': 0.0009765625, 'config/reg_lambda': 5.525802807180917, 'experiment_tag': 'exp', 'time_total_s': 1.

...

[flaml.tune.tune: 09-25 13:02:02] {197} INFO - result: {'pred_time': 1.3996768080225415e-05, 'wall_clock_time': 1778.6427128314972, 'metric_for_logging': {'pred_time': 1.3996768080225415e-05}, 'val_loss': 0.21505473874289463, 'trained_estimator': <flaml.automl.model.LRL1Classifier object at 0x7fe5340d3b50>, 'training_iteration': 0, 'config': {'C': 3.037791154364551}, 'config/C': 3.037791154364551, 'experiment_tag': 'exp', 'time_total_s': 26.10639715194702}
[flaml.tune.tune: 09-25 13:02:02] {197} INFO - result: {'pred_time': 1.3996768080225415e-05, 'wall_clock_time': 1778.6427128314972, 'metric_for_logging': {'pred_time': 1.3996768080225415e-05}, 'val_loss': 0.21505473874289463, 'trained_estimator': <flaml.automl.model.LRL1Classifier object at 0x7fe5340d3b50>, 'training_iteration': 1, 'config': {'C': 3.037791154364551}, 'config/C': 3.037791154364551, 'experiment_tag': 'exp', 'time_total_s': 26.10824418067932}
[flaml.automl.logger: 09-25 13:02:02] {2391} INFO -  at 1778.6s,	estimator lrl

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.tune.tune: 09-25 13:02:04] {197} INFO - result: {'pred_time': 6.640528884026332e-05, 'wall_clock_time': 1780.4518666267395, 'metric_for_logging': {'pred_time': 6.640528884026332e-05}, 'val_loss': 0.2143001480895034, 'trained_estimator': <flaml.automl.model.ExtraTreesEstimator object at 0x7fe534076ac0>, 'training_iteration': 0, 'config': {'n_estimators': 455, 'max_features': 0.0645416003203881, 'max_leaves': 374, 'criterion': 'gini'}, 'config/n_estimators': 455, 'config/max_features': 0.0645416003203881, 'config/max_leaves': 374, 'config/criterion': 'gini', 'experiment_tag': 'exp', 'time_total_s': 1.7984724044799805}
[flaml.tune.tune: 09-25 13:02:04] {197} INFO - result: {'pred_time': 6.640528884026332e-05, 'wall_clock_time': 1780.4518666267395, 'metric_for_logging': {'pred_time': 6.640528884026332e-05}, 'val_loss': 0.2143001480895034, 'trained_estimator': <flaml.automl.model.ExtraTreesEstimator object at 0x7fe534076ac0>, 'training_iteration': 1, 'config': {'n_estimators': 455, '

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarnin

[flaml.automl.logger: 09-25 13:03:36] {2574} INFO - ensemble: StackingClassifier(estimators=[('xgboost',
                                <flaml.automl.model.XGBoostSklearnEstimator object at 0x7fe6d5b622e0>),
                               ('extra_tree',
                                <flaml.automl.model.ExtraTreesEstimator object at 0x7fe5340bcfa0>),
                               ('rf',
                                <flaml.automl.model.RandomForestEstimator object at 0x7fe5340bc370>),
                               ('xgb_limitdepth',
                                <flaml.automl.model.XGBoostLimitDepthEstimator object at 0x7fe5340bc700>),
                               ('lgbm',
                                <flaml.automl.model.LGBMEstimator object at 0x7fe5340bc3d0>),
                               ('lrl1',
                                <flaml.automl.model.LRL1Classifier object at 0x7fe5340bc7f0>),
                               ('catboost',
                                <fl

In [19]:
y_pred = automl.predict(X_test)
accuracy_score(y_test,y_pred) 

0.9888641425389755

In [20]:
print(automl.model.estimators, automl.model.final_estimator)

AttributeError: 'StackingClassifier' object has no attribute 'estimator'

In [21]:
print(automl.best_estimator)
print(automl.best_config)

xgboost
{'n_estimators': 139, 'max_leaves': 136, 'min_child_weight': 0.27084276373818733, 'learning_rate': 0.0484091845623208, 'subsample': 0.827351358517848, 'colsample_bylevel': 0.35468713776971533, 'colsample_bytree': 0.6256302030953867, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.3630887969527579}



## TPOT

In [11]:
from tpot import TPOTClassifier

In [12]:
%%time
pipeline_optimizer = TPOTClassifier(verbosity=3, max_time_mins=time_min, random_state=dagstuhl_seed,  config_dict='TPOT light')
pipeline_optimizer.fit(X_train, y_train)

19 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]

_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.
_pre_test decorator: _random_mutation_operator: num_test=1 Solver lbfgs supports only dual=False, got dual=True.
_pre_test decorator: _random_mutation_operator: num_test=2 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Negative values in data passed to MultinomialNB (input X).
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_tes

TPOTClassifier(config_dict='TPOT light', max_time_mins=30, random_state=23372,
               verbosity=3)

In [13]:
print(pipeline_optimizer.score(X_test, y_test))

0.9991648106904232


In [14]:
pipeline_optimizer.export('tpot_exported_pipeline.py')

In [17]:
pipeline_optimizer.fitted_pipeline_

Pipeline(steps=[('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=9, p=1, weights='distance'))])